In [ ]:
# parts 1 through 3

In [3]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

In [9]:
roi_df = pd.read_csv('roi_company1.csv')
roi_df

,Platform,Tier,LowerBound,UpperBound,ROI
0,Print,1,0.0,0.7,0.0345
1,Print,2,0.7,2.7,0.0305
2,Print,3,2.7,4.7,0.0276
3,Print,4,4.7,inf,0.0234
4,TV,1,0.0,4.3,0.0608
5,TV,2,4.3,inf,0.0503
6,SEO,1,0.0,3.6,0.0288
7,SEO,2,3.6,inf,0.0269
8,AdWords,1,0.0,2.0,0.0419
9,AdWords,2,2.0,2.3,0.0358


In [10]:
m = Model('marketing allocation')
m.Params.LogToConsole = 1
m.Params.OutputFlag = 1

Set parameter LogToConsole to value 1
Set parameter OutputFlag to value 1


In [15]:
# global tunables

TOTAL_BUDGET = 10.0   # Total marketing budget in millions
PLATFORM_MAX_SPEND = 3.0  # Per-platform maximum investment (M$)

In [16]:
# break out individual tiers

# Unique platforms
platforms = roi_df["Platform"].unique().tolist()

# Tiers per platform
tiers = {
    p: roi_df.loc[roi_df["Platform"] == p, "Tier"].tolist()
    for p in platforms
}

# ROI values per (platform, tier)
rois = {
    (row.Platform, int(row.Tier)): row.ROI
    for _, row in roi_df.iterrows()
}

# Tier widths (UpperBound - LowerBound, with global cap for 'inf')
widths = {}
for _, row in roi_df.iterrows():
    upper = str(row.UpperBound).lower()
    if upper == "inf":
        # last tier: allow up to TOTAL_BUDGET beyond the lower bound
        w = TOTAL_BUDGET - float(row.LowerBound)
    else:
        w = float(row.UpperBound) - float(row.LowerBound)
    widths[(row.Platform, int(row.Tier))] = max(w, 0.0)  # safety clamp

# --- Sanity check ---
print("Platforms:", platforms)
print("Example ROI list for Print:", [rois[("Print", t)] for t in tiers["Print"]])
print("Tier widths for Print:", [widths[("Print", t)] for t in tiers["Print"]])


Platforms: ['Print', 'TV', 'SEO', 'AdWords', 'Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter', 'Email']
Example ROI list for Print: [0.0345, 0.0305, 0.0276, 0.0234]
Tier widths for Print: [0.7, 2.0, 2.0, 5.3]


In [17]:
# decision variables

x = {}  # decision variables: amount (M$) invested in platform p, tier t

for p in platforms:
    for t in tiers[p]:
        ub = widths[(p, t)]
        var_name = f"x_{p}_T{t}"           # e.g. x_Print_T1, x_TV_T3, etc.
        x[p, t] = m.addVar(
            lb=0,
            ub=ub,
            vtype=GRB.CONTINUOUS,
            name=var_name
        )

m.update()
print(f"Added {len(x)} decision variables across {len(platforms)} platforms.")


Added 28 decision variables across 10 platforms.


In [18]:
# Objective: maximize total return (ROI × investment)
objective = quicksum(rois[(p, t)] * x[p, t] for p in platforms for t in tiers[p])
m.setObjective(objective, GRB.MAXIMIZE)

In [19]:
# constraints

# 1. Total budget constraint
m.addConstr(
    quicksum(x[p, t] for p in platforms for t in tiers[p]) <= TOTAL_BUDGET,
    name="Total_Budget"
)

# 2. Per-platform maximum spend constraint
for p in platforms:
    m.addConstr(
        quicksum(x[p, t] for t in tiers[p]) <= PLATFORM_MAX_SPEND,
        name=f"PlatformMax_{p}"
    )

# 3. Print + TV ≤ Facebook + Email
m.addConstr(
    quicksum(x[p, t] for p in ["Print", "TV"] for t in tiers[p]) <=
    quicksum(x[p, t] for p in ["Facebook", "Email"] for t in tiers[p]),
    name="PrintTV_vs_FacebookEmail"
)

# 4. Social media ≥ 2 × (SEO + AdWords)
social_media = ["Facebook", "LinkedIn", "Instagram", "Snapchat", "Twitter"]
search_media = ["SEO", "AdWords"]

m.addConstr(
    quicksum(x[p, t] for p in social_media for t in tiers[p]) >=
    2 * quicksum(x[p, t] for p in search_media for t in tiers[p]),
    name="Social_vs_Search"
)

# 5. Tier spending order / upper-bound concavity
# This behavior is guaranteed by concavity: higher-ROI tiers will fill first
# under linear maximization, so we do not need explicit tier-linking constraints.

print("All constraints added successfully.")


All constraints added successfully.


In [20]:
# optimize and clean output 

m.optimize()

if m.status == GRB.OPTIMAL:
    print("\n=== Optimization Results ===")

    # Compute and display results by platform and tier
    total_spend = 0.0
    total_return = 0.0
    platform_summary = {}

    print(f"\n{'Platform':<12} {'Tier':<6} {'Investment (M$)':>18} {'ROI':>10} {'Return (M$)':>14}")
    print("-" * 60)

    for p in platforms:
        platform_invest = 0.0
        platform_return = 0.0
        for t in tiers[p]:
            invest = x[p, t].X
            if invest > 1e-6:
                roi = rois[(p, t)]
                ret = invest * roi
                total_spend += invest
                total_return += ret
                platform_invest += invest
                platform_return += ret
                print(f"{p:<12} {t:<6} {invest:>18.2f} {roi:>10.4f} {ret:>14.2f}")
        platform_summary[p] = (platform_invest, platform_return)

    # Platform totals
    print("\n=== Platform Totals ===")
    print(f"{'Platform':<12} {'Total Invest (M$)':>20} {'Total Return (M$)':>20} {'ROI (%)':>10}")
    print("-" * 65)
    for p, (inv, ret) in platform_summary.items():
        if inv > 1e-6:
            roi_pct = (ret / inv) * 100
            print(f"{p:<12} {inv:>20.2f} {ret:>20.2f} {roi_pct:>10.2f}")

    # Global totals
    overall_roi = (total_return / total_spend) * 100 if total_spend > 0 else 0
    print("\n=== Overall Summary ===")
    print(f"Total Spend:     {total_spend:.2f} M$")
    print(f"Total Return:    {total_return:.2f} M$")
    print(f"Average ROI:     {overall_roi:.2f}%")

else:
    print("Model did not reach optimal status.")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 24.5.0 24F74)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 28 columns and 87 nonzeros
Model fingerprint: 0xbd9c74d5
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e-02, 6e-02]
  Bounds range     [3e-01, 8e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.03s
Presolved: 10 rows, 25 columns, 75 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0800000e-01   1.700000e+01   0.000000e+00      0s
       2    5.4364000e-01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.04 seconds (0.00 work units)
Optimal objective  5.436400000e-01

=== Optimization Results ===

Platform     Tier      Investment (M$)        ROI    Return (M$)
----------------------------------------------------